In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import en_core_web_lg
import csv
import regex as re
import os
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import corpora

In [2]:
path = './DataUCSB/'
list_of_files = []

for root, dirs, files in os.walk(path):
    for file in files:
        list_of_files.append(os.path.join(root,file))

len(list_of_files)

135

In [3]:
filepath = './DataUCSB/address-before-joint-session-the-congress-the-state-the-union-16.csv'
speeches = []
for file in list_of_files:
    with open(filepath, 'r') as read_obj: # read csv file as a list of lists
      csv_reader = csv.reader(read_obj) # pass the file object to reader() to get the reader object
      speechList = sum(list(csv_reader), []) # Pass reader object to list() to get a list of lists (matrix)
                                            # sum(list, []) flattens 2D matrix into a vector
      speech = ''.join(speechList)
    speeches.append(speech)

<A HREF="https://regexr.com/">RegExr regex tester</A>

In [ ]:
#Load SpaCy English Model
nlp = en_core_web_lg.load()

#Tags to remove
extags = ['PRON','CCONJ','PUNCT','PART','DET','ADP','NUM','SYM','SPACE']

tokens=[]
#SpaCy tokenization + lemmatization + lowercase
for speech in nlp.pipe(speeches):
    scr_tok = [token.lemma_.lower() for token in speech if token.pos_ not in extags and not token.is_stop and token.is_alpha]
    tokens.append(scr_tok)
tokens

In [17]:
nlp = spacy.load("en_core_web_lg")
doc = [nlp(speech) for speech in speeches]

In [8]:
#Tags to remove
extags = ['PRON','CCONJ','PUNCT','PART','DET','ADP','NUM','SYM','SPACE']
tokens = [ (token.lemma_.lower() for token in speech if token.pos_ not in extags and not token.is_stop and token.is_alpha) for speech in speeches ]

dictionary = Dictionary(tokens)
#dictionary.filter_extremes(no_below=10, no_above=0.5)
#Convert token counts into bag of words (BoW) corpus
corpus = [dictionary.doc2bow(token) for token in tokens]

AttributeError: 'str' object has no attribute 'pos_'

In [51]:
corpus

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [33]:
# Make a index to word dictionary.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

#Train the topic model

model = LdaModel(corpus=corpus, id2word=id2word,iterations=400, num_topics=10)
top_topics = list(model.top_topics(corpus))
pprint(top_topics)

KeyError: 0

In [ ]:
# Not working

#Construct and train unsupervised LDA model + Determine optimal amount of topics
umtopics, umscore = [], []
cvtopics, cvscore = [], []
#Compute coherence score using C_umass:
for i in range(1,20,1):
    model = LdaMulticore(corpus=corpus, id2word=dict, iterations=50, num_topics=i, workers=4, passes=10, random_state=47)
    #LdamultiCore uses multiple cores to speed up model training, use with caution if you have a weaker PC! (Find your max number of cores with ctrl+shift+esc, under CPU)
    cm = CoherenceModel(model=model, corpus=corpus, dictionary=dict, coherence='u_mass')

    umtopics.append(i)
    umscore.append(cm.get_coherence())
#Compute coherence score using C_v: 
for i in range (1,20,1):
    model = LdaMulticore(corpus=corpus, id2word=dict, iterations=10, num_topics=i, workers = 4, passes=10, random_state=47)
    cm = CoherenceModel(model=model, texts = tokens, corpus=corpus, dictionary=dict, coherence='c_v')

    cvtopics.append(i)
    cvscore.append(cm.get_coherence())
#The difference in coherence score measures is the method in which the text is segmented and probability is calculated
